# <center>[⚔️ Блендинг - смешай и скор вырастет](https://stepik.org/lesson/825510/)</center>

### Оглавление ноутбука

<img src='../images/blending.jpg' align="right" width="750" height="750" />
<br>

<p><font size="3" face="Arial" font-size="large"><ul type="square">
    
<li><a href="#c1">😺🚀 Обучаем CatBoost </a></li>
<li><a href="#c1">🦄🎳 Обучаем LightGBM </a></li>
<li><a href="#c1">👽🔱 Обучаем XGBoost </a></li>
<li><a href="#c1">🐲 Блендинг и принципы блендинга </a></li>
<li><a href="#6">🧸 Выводы и заключения</a>

</li></ul></font></p>

    

<div class="alert alert-info">

* Основная идея данной техники заключается в том, чтобы взять от каждого алгоритма лучшее и совместить несколько разных ML моделей в одну.
* За счет такого объединения увеличивается обобщающая способность финальной модели и качество улучшается.
* Помимо этого ваша модель становится более стабильной, что позволяет не слететь на приватном лидерборде.
* Особенно хорошо накидывает блендинг, если смешиваемые **модели имеют разную природу**: например, нейронные сети, KNN и решающие деревья, в этом случае они выучивают разные зависимости и хорошо дополняют друг друга.

In [3]:
!pip3 install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.1 MB/s eta 0:00:00


## Импортируем библиотеки

In [ ]:
import numpy as np
import pandas as pd

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import warnings
warnings.filterwarnings("ignore")

In [5]:
# Модели для смешивания
import lightgbm as lgbm
import xgboost as xgb
import catboost as cb

## Считываем данные

In [ ]:
data = pd.read_csv("../data/quickstart_train.csv")

### Заменим категориальные признаки на числовые значения
cat_cols = ["model", "car_type", "fuel_type"]
for col in cat_cols:
    data[col] = data[col].replace(np.unique(data[col]), np.arange(data[col].nunique()))
    data[col] = data[col].astype("category")

data.head()

### Разделим выборку на валидационную и обучающую

In [ ]:
cols2drop = ["car_id", "target_reg", "target_class"]

X_train, X_val, y_train, y_val = train_test_split(
    data.drop(cols2drop, axis=1),
    data["target_reg"],
    test_size=0.25,
    stratify=data["target_class"],
    random_state=42,
)
print(X_train.shape, X_val.shape)

# <center> Обучим три модели для смешивания - блендинга

## 😺🚀 Обучаем **`CatBoost`**

In [ ]:
params_cat = {
    "n_estimators": 1500,
    "learning_rate": 0.03,
    "depth": 3,
    "use_best_model": True,
    "cat_features": cat_cols,
    "text_features": [],
    # 'train_dir' : '/path/to/catboost/model',
    "border_count": 64,
    "l2_leaf_reg": 1,
    "bagging_temperature": 2,
    "rsm": 0.5,
    "loss_function": "RMSE",  # Не определена для регрессии
    # 'auto_class_weights' : 'Balanced', # Не определен для регрессии
    "random_state": 42,
    "custom_metric": ["MAE", "MAPE"],
}

cat_model = cb.CatBoostRegressor(**params_cat)

In [ ]:
cat_model.fit(
    X_train,
    y_train,
    verbose=100,
    eval_set=(X_val, y_val),
    early_stopping_rounds=150,
)

In [ ]:
print('MSE (catboost) : ', mean_squared_error(cat_model.predict(X_val), y_val).round(3))

In [ ]:
# Сравним с бейзлайном в виде среднего значения
mean_squared_error(np.ones(len(y_val)) * y_val.mean(), y_val).round(3)

In [ ]:
submit = pd.DataFrame({"target": cat_model.predict(X_val).reshape(-1)})
submit.to_csv("../tmp_data/catboost_preds.csv", index=False)
submit.head()

## 🦄🎳 Обучаем `LightGBM`

In [2]:
params_lgbm = {
    "num_leaves": 200,
    "n_estimators": 1500,
    # "max_depth": 7,
    "min_child_samples": 2073,
    "learning_rate": 0.0051,
    "min_data_in_leaf": 10,
    "feature_fraction": 0.99,
    "categorical_feature": cat_cols,
    'reg_alpha' : 5.0,
    'reg_lambda' : 5.0,
}

lgbm_model = lgbm.LGBMRegressor(**params_lgbm)

NameError: name 'cat_cols' is not defined

In [ ]:
lgbm_model.fit(
    X_train,
    y_train,
    eval_set=(X_val, y_val),
    early_stopping_rounds=100,
    verbose=150,
)

In [ ]:
print('MSE (lgb) : ', mean_squared_error(lgbm_model.predict(X_val), y_val).round(3))

MSE (lgb) :  139.132


In [ ]:
submit = pd.DataFrame({"target": lgbm_model.predict(X_val).reshape(-1)})
submit.to_csv("../tmp_data/lgbm_preds.csv", index=False)
submit.head()

,target
0,33.660865
1,42.797476
2,34.142549
3,64.390769
4,66.186459


## 👽🔱 Обучаем `XGBoost`

In [ ]:
# xgboost не умеет работать с категориальными признаками, так что нужно сделать ohe
X_train = pd.get_dummies(X_train, columns=["car_type", "fuel_type", "model"])
X_val = pd.get_dummies(X_val, columns=["car_type", "fuel_type", "model"])
X_train.shape

(1752, 43)

In [ ]:
params_xgb = {
    "eta": 0.05,
    "max_depth": 5,
    "subsample": 0.7,
    "colsample_bytree": 0.7,
    'gamma': .01,
    'reg_lambda' : 0.1,
    'reg_alpha' : 0.5,
    "objective": "reg:linear",
    "eval_metric": "mae",
    'tree_method' : 'hist', # Supported tree methods for cat fs are `gpu_hist`, `approx`, and `hist`.
    'enable_categorical' : True

}

xgb_model = xgb.XGBRegressor(**params_xgb)

In [ ]:
xgb_model.fit(
    X_train,
    y_train,
    eval_set=[(X_train, y_train), (X_val, y_val)],
    early_stopping_rounds=100,
    verbose=25,
)
print('best_iteration', xgb_model.best_iteration)

[07:20:08] WARNING: ../src/objective/regression_obj.cu:213: reg:linear is now deprecated in favor of reg:squarederror.
[0]	validation_0-mae:42.24252	validation_1-mae:42.20440
[25]	validation_0-mae:12.70880	validation_1-mae:12.98292
[50]	validation_0-mae:7.79993	validation_1-mae:9.14011
[75]	validation_0-mae:6.97940	validation_1-mae:8.76001
[99]	validation_0-mae:6.55026	validation_1-mae:8.74984
best_iteration 90


In [ ]:
print('MSE (xgboost) : ', mean_squared_error(xgb_model.predict(X_val), y_val).round(3))

MSE (xgboost) :  137.036


In [ ]:
submit = pd.DataFrame({"target": xgb_model.predict(X_val).reshape(-1)})
submit.to_csv("../tmp_data/xgb_preds.csv", index=False)
submit.head()

,target
0,32.807278
1,43.922146
2,33.644127
3,60.386654
4,69.922684


# <center> 😈 А теперь блендим модели!

* __Быстрый и простой способ сбледнить - это усреднить ответы!__ <br>
* __Но мы сразу покажем смешивание с весами моделей - взвешивание__

In [ ]:
cb_model  = pd.read_csv("../tmp_data/catboost_preds.csv")["target"]  # 132.051 Лучшая (ставим ей больший вес)
xgb_model = pd.read_csv("../tmp_data/xgb_preds.csv")["target"]       # 137.844 Средняя
lgb_model = pd.read_csv("../tmp_data/lgbm_preds.csv")["target"]      # 139.132 Худшая (ставим ей меньший вес)

In [ ]:
# score1 > score2 > score3 : w1 > w2 > w3
ensemble = cb_model * 0.50 + xgb_model * 0.35 + lgb_model * 0.15

In [ ]:
print('MSE (ensemble) :', mean_squared_error(ensemble, y_val).round(3))

MSE (ensemble) : 131.85


In [ ]:
round((100*(132.051 - 131.080)/132.051), 1)

0.7

# <center> 🥳  Ура, скор улучшился!
    
<div class="alert alert-info">
    
**Как подбирать веса?**
- Опираясь на скоры на лидерборде
- По локальной валидации
- Ставить веса пропорциональны скору

### Если хочется кодом, то вот более универсальный способ

In [ ]:
weights = {"catboost": 0.5, "lgbm": 0.15, "xgb": 0.35}

In [ ]:
import os

preds = pd.DataFrame()

# соберем единый датафрейм из наших предсказаний
for model_name in ["catboost", "lgbm", "xgb"]:

    path = os.path.join("../tmp_data/", f"{model_name}_preds.csv")
    now = pd.read_csv(path).reset_index()

    now["model"] = model_name
    now["target"] *= weights[model_name]
    preds = pd.concat([preds, now])

preds.head()

,index,target,model
0,0,16.493947,catboost
1,1,23.804264,catboost
2,2,17.544528,catboost
3,3,30.981306,catboost
4,4,35.728063,catboost


In [ ]:
preds["model"].unique()

array(['catboost', 'lgbm', 'xgb'], dtype=object)

In [ ]:
ensemble = preds.groupby("index")["target"].agg("sum")
mean_squared_error(ensemble, y_val)

131.84991442662573

# 🐲 Принципы блендинга (просто и логично)

<div class="alert alert-info">

* 🦑 Не бленди, пока не выжал максимум из моделей по отдельности
* 🐳 Чем различнее и сильнее модели, тем эффективнее блендинг (⚠️ эффективность)
* 🐙 При равной точности, ансабль побеждает соло-модель на привате (⚠️ стабильность)
* 🦐 Чем раньше проверишь эффект от блендинга, тем эффективне будет стратегия
* 🐠 Блендить можно с разными весами, пропорционально скорам или разным фичам
* 🐋 Блендинг по фолдам и чекпоинтам обучения - это тоже блендинг
* 🐡 Блендинг по сидам - это стабилизирующий блендинг
* 🐬 Против блендинга только больший блендинг

<div class="alert alert-info">
    
__забегая вперёд__
    
* 🦈 Стекинг сильнее блендинга, но капризнее
* 🎣 Стекать можно с фичами
* 🎏 Стекинг бывает разных уровней
* 🐉 Не можешь больше стекать - блендись с решением сокомандников

# 🧸 Выводы и заключения

<div class="alert alert-info">

* Блендинг - это сильный инструмент, который зачастую неплохо поднимает качество моделей.
* При этом само смешивание провести можно вообще в одну строчку, просто взяв среднее моделей и взвесив
* `ensemble = model1 * w1 + model2 * w2 + model3 * w3 + ...`